# 特征提取脚本说明（feature_extract.py）

## 目标
基于已有的Excel数据（如 `PR_info_add_conversation.xlsx`、`PR_features.xlsx`、`author_features.xlsx`、`PR_comment_info.xlsx`），生成用于建模的综合特征文件 `PR_extracted_features.xlsx`。

## 输入与输出
- 输入目录：`yii2/`
- 输入文件：
  - `PR_info_add_conversation.xlsx`
  - `PR_features.xlsx`
  - `author_features.xlsx`
  - `PR_comment_info.xlsx`
- 输出文件：
  - `PR_extracted_features.xlsx`（位于 `yii2/`）

## 主要处理步骤
1. 过滤关闭状态的PR（`state == 'closed'`）
2. 统一时间列时区并转为naive datetime
3. 从 `PR_info_add_conversation.xlsx` 中保留关键列（编号与时间）并计算 `last_pr_update`
4. 从 `PR_features.xlsx` 合并标题长度、正文长度、文件增删改统计
5. 从 `author_features.xlsx` 合并作者活跃度（`changes_per_week`）与合入比例（`merge_proportion`）
6. 使用 `PR_comment_info.xlsx` 计算 `last_comment_update`（首条/最新评论相对PR创建的小时差的最大值）
7. 按 `number` 排序并写出 `PR_extracted_features.xlsx`

## 运行方式
- 直接运行脚本（默认从 `yii2/` 读取，输出到 `yii2/`）：
```bash
python ML-lab1/feature_extract.py
```

## 作用
- 生成的 `PR_extracted_features.xlsx` 可直接用于 Task1/Task2 的模型训练与评估。


In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('yii2')
assert DATA_DIR.exists(), f"数据目录不存在: {DATA_DIR.resolve()}"

# 1) 读取数据
a_pr = pd.read_excel(DATA_DIR / 'PR_info_add_conversation.xlsx', engine='openpyxl')
feat = pd.read_excel(DATA_DIR / 'PR_features.xlsx', engine='openpyxl')
auth = pd.read_excel(DATA_DIR / 'author_features.xlsx', engine='openpyxl')
cmts = pd.read_excel(DATA_DIR / 'PR_comment_info.xlsx', engine='openpyxl')
print('加载完成:', len(a_pr), len(feat), len(auth), len(cmts))

# 2) 过滤关闭PR
pr = a_pr[a_pr['state'] == 'closed'].copy()

# 3) 统一时间列并转为naive
datetime_cols = ['created_at', 'updated_at', 'merged_at', 'closed_at']
for col in datetime_cols:
    pr[col] = pd.to_datetime(pr[col], utc=True)
    pr[col] = pr[col].dt.tz_localize(None)

# 4) 选择关键列并计算 last_pr_update
pr = pr[["number", "created_at", "updated_at", "merged_at", "closed_at", "merged", "additions", "deletions"]]
pr['last_pr_update'] = (pr['updated_at'] - pr['created_at']).dt.total_seconds() / 3600

# 5) 合并 PR_features
use_feat_cols = ['number', 'title_length', 'body_length', 'files_added', 'files_deleted', 'files_updated']
pr = pr.merge(feat[use_feat_cols], on='number', how='left')

# 6) 合并 author_features
use_auth_cols = ['number', 'changes_per_week', 'merge_proportion']
pr = pr.merge(auth[use_auth_cols], on='number', how='left')

# 7) 计算 last_comment_update（按PR分组的最大评论间隔）
cmts['updated_at'] = pd.to_datetime(cmts['updated_at'], utc=True).dt.tz_localize(None)

pr_with_cmts = pd.merge(pr, cmts, left_on='number', right_on='belong_to_PR', how='left', suffixes=('_pr', '_cmt'))

if 'updated_at_cmt' in pr_with_cmts.columns and 'created_at_pr' in pr_with_cmts.columns:
	delta = (pr_with_cmts['updated_at_cmt'] - pr_with_cmts['created_at_pr'])
elif 'updated_at_y' in pr_with_cmts.columns and 'created_at_x' in pr_with_cmts.columns:
	delta = (pr_with_cmts['updated_at_y'] - pr_with_cmts['created_at_x'])
else:
	raise KeyError('找不到评论或PR时间列，请检查输入数据列名')
pr_with_cmts['last_comment_update'] = delta.dt.total_seconds() / 3600
last_cmt = pr_with_cmts.groupby('number')['last_comment_update'].max()
pr = pr.merge(last_cmt, on='number', how='left')

# 8) 排序 & 保存
pr = pr.sort_values('number')
out_path = DATA_DIR / 'PR_extracted_features.xlsx'
pr.to_excel(out_path, index=False)
print('已写出:', out_path.resolve())
print('列:', list(pr.columns)[:12], '...')
print('样例行数/列数:', pr.shape)



加载完成: 8040 8040 8040 25643
已写出: D:\Develop\ML\Lab1\ML_2025fall_njuse\ML-lab1\yii2\PR_extracted_features.xlsx
列: ['number', 'created_at', 'updated_at', 'merged_at', 'closed_at', 'merged', 'additions', 'deletions', 'last_pr_update', 'title_length', 'body_length', 'files_added'] ...
样例行数/列数: (7957, 17)
